https://lectures.quantecon.org/py/aiyagari.html

### Household's problem

\begin{align}
            \max  \quad & U = E_{0} \sum_{t=0}^{\infty} \beta^{t} u \left( c_{t} \right) \\
\text{subject to} \quad & c_{t} + a_{t+1} = w \cdot z_{t} + \left( 1+r \right) a_{t} \\
                        & c_{t} \geq 0 \\
                        & a_{t+1} \geq -b
\end{align}

Exogenous process for $z_{t}$ follows a finite state Markov chain with given stochastic matrix $P$

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from time import time
from bisect import *

from scipy import sparse
from scipy.sparse.linalg import spsolve, eigs

from scipy import optimize
from scipy.interpolate import InterpolatedUnivariateSpline

ImportError: cannot import name 'bisect_left' from 'bisect' (C:\Users\Marcin\Dropbox\Classes\Applied_Macroeconomics\bisect.py)

In [ ]:
# Household labor efficiency process
z_grid = np.array([0.1, 1.0])
z_len = 2

P = np.array([[0.9, 0.1], 
              [0.1, 0.9]])

In [ ]:
# Stationary distribution of states

D, V = np.linalg.eig(P.T)

col = np.where(np.abs(D-1) < 1e-3)[0][0]

v = V[:, col]

P_bar = v/sum(v)
print('P_bar =', P_bar)

N = P_bar @ z_grid.T
print('N =', N)

In [ ]:
# Parameters
α = 0.33
β = 0.96
δ = 0.05
σ = 1
A = 1

In [ ]:
def Capital_Demand(r):
    return (α/(r+δ))**(1/(1-α)) * N

def Wage(r):
    K = Capital_Demand(r)
    return (1-α) * (K/N)**α

In [ ]:
rr = np.linspace(0, 0.1, 100) #1/β-1

plt.plot(Capital_Demand(rr), rr, label='Demand for capital')
plt.xlabel('Aggregate capital stock $K$')
plt.ylabel('Real interest rate $r$')
plt.legend()
plt.show()

In [ ]:
a_min = 0
a_max = 20
a_len = 1+20

a_grid = np.linspace(a_min, a_max, a_len)
# a_grid = np.geomspace(a_min+1, a_max+1, a_len)-1

In [ ]:
# Initial guess for r
r = 0.02
R = 1+r
w = Wage(r)
w

In [ ]:
# Endogenous Gridpoint Method

t0 = time()

c_opt = np.zeros((z_len, a_len))
c_new = np.zeros((z_len, a_len))

a_new = np.zeros((z_len, a_len))
a_opt = np.zeros((z_len, a_len))

# Initial guess for c_opt
for j, z in enumerate(z_grid):
    c_opt[j, :] = r*a_grid[:] + z*w

norm_pol = 1
it = 0
maxit = 5000

while norm_pol > 1e-6*(1-β) and it < maxit:
    ERHS = P @ (β*R * c_opt**(-σ))
    IERHS = ERHS**(-1/σ)
    for j, z in enumerate(z_grid):
        c_end = IERHS[j, :]
        a_end = (a_grid[:] + c_end - z*w) / R
    
        c_new[j, :] = InterpolatedUnivariateSpline(a_end, c_end, k=1)(a_grid)
        
        # Binding constraint
        bind = np.where(a_grid < a_end[0])
        c_new[j, bind] = R*a_grid[bind] + z*w - a_min
        
        a_new[j, :] = R*a_grid + z*w - c_new[j, :]
    
    norm_pol = np.max(np.abs(c_opt - c_new))
    c_opt[:] = c_new[:]
    a_opt[:] = a_new[:]
    
    it = it + 1
    
    if it % 100 == 0:
        print('Iteration', it, '\t polfun norm', norm_pol)

t1 = time()

print('')
print('Elapsed time is', t1-t0, 'seconds.')
print('Converged after', it, 'iterations.')
print('Policy function L_inf norm is', norm_pol)

In [ ]:
a_EGM = np.zeros((z_len, a_len))
c_EGM = np.zeros((z_len, a_len))

for j, z in enumerate(z_grid):
    c_EGM[j, :] = c_opt[j, :]
    a_EGM[j, :] = R*a_grid + z*w - c_EGM[j, :]

# plt.subplots(figsize=(4,3))
plt.plot(a_grid, a_EGM[1, :], lw=2, label='employed')
plt.plot(a_grid, a_EGM[0, :], lw=2, label='unemployed')
plt.plot(a_grid, a_grid, 'k-', lw=1)
plt.title('Policy function: assets')
plt.xlabel('Current period assets $a$')
plt.legend(frameon=False)
plt.tight_layout()
# plt.savefig('PE_polfun_A.pdf')
plt.show()

# plt.subplots(figsize=(4,3))
plt.plot(a_grid, c_EGM[1, :], lw=2, label='employed')
plt.plot(a_grid, c_EGM[0, :], lw=2, label='unemployed')
plt.title('Policy function: consumption')
plt.xlabel('Current period assets $a$')
plt.legend(frameon=False)
plt.tight_layout()
# plt.savefig('PE_polfun_C.pdf')
plt.show()

# plt.subplots(figsize=(4,3))
plt.plot(a_grid, a_EGM[1, :]-a_grid, lw=2)
plt.plot(a_grid, np.zeros(len(a_grid)), 'k-', lw=1)
plt.title('Employed: $a_{t+1} - a_{t}$')
plt.xlabel('Current period assets $a$')
plt.tight_layout()
# plt.savefig('PE_polfun_AA.pdf')
plt.show()

In [ ]:
dens_mult = 1

In [ ]:
a_grid_dens = np.linspace(a_min, a_max, dens_mult*a_len)
a_len_dens = len(a_grid_dens)
index_dens = range(a_len_dens)

pos_l = np.zeros((z_len, a_len_dens), dtype=int)
pos_h = np.zeros((z_len, a_len_dens), dtype=int)

val_l = np.zeros((z_len, a_len_dens))
val_h = np.zeros((z_len, a_len_dens))

a_EGM = np.zeros((z_len, a_len_dens))

for j, z in enumerate(z_grid):
    a_EGM[j, :] = InterpolatedUnivariateSpline(a_grid, a_opt[j, :], k=1)(a_grid_dens)

for j, z in enumerate(z_grid):
    for i, a in enumerate(a_grid_dens):
        pos_h[j, i] = min(bisect(a_grid_dens, a_EGM[j, i]), a_len_dens-1)
        pos_l[j, i] = max(pos_h[j, i]-1, 0)

        val_h[j, i] = min(1, ((a_EGM[j, i] - a_grid_dens[pos_l[j, i]]) 
                              / (a_grid_dens[pos_h[j, i]] - a_grid_dens[pos_l[j, i]])) )
        val_l[j, i] = 1 - val_h[j, i]

G_0 = sparse.coo_matrix( ( np.concatenate((val_l[0, :], val_h[0, :])), 
                          ( np.concatenate((index_dens, index_dens)), 
                            np.concatenate((pos_l[0, :], pos_h[0, :])) ) ), 
                        shape=(a_len_dens, a_len_dens) ).tocsr()

G_1 = sparse.coo_matrix( ( np.concatenate((val_l[1, :], val_h[1, :])), 
                          ( np.concatenate((index_dens, index_dens)), 
                            np.concatenate((pos_l[1, :], pos_h[1, :])) ) ), 
                        shape=(a_len_dens, a_len_dens) ).tocsr()

G = sparse.vstack( (sparse.kron(P[0, :], G_0), 
                    sparse.kron(P[1, :], G_1)) )

val, vec = eigs(G.transpose(), k=1, which='LR', return_eigenvectors=True)

stat_EGM = np.real(vec.flat)
stat_EGM = stat_EGM / np.sum(stat_EGM)

K_sup = a_grid_dens @ stat_EGM[a_len_dens:] + a_grid_dens @ stat_EGM[:a_len_dens]

print('Capital supplied =', K_sup)
print('')

# plt.subplots(figsize=(4,3))
plt.plot(a_grid_dens, stat_EGM[a_len_dens:], lw=2, label='employed')
plt.plot(a_grid_dens, stat_EGM[:a_len_dens], lw=2, label='unemployed')
plt.title('Wealth distribution')
plt.xlabel('Current period assets $a$')
# plt.xlim(-5, 105)
plt.legend(frameon=False)
plt.tight_layout()
# plt.savefig('PE_dist.pdf')
plt.show()

In [ ]:
def K_sup(r, dens_mult=1):
    K_dem = (α*A/(r+δ))**(1/(1-α)) * N
    w = (1-α) * (K_dem/N)**α
    R = 1+r
    
#     print('Interest rate pa =', r) #100*((1+r)**8-1)
#     print('Capital demanded =', K_dem)
    
    # Endogenous Gridpoint Method

#     t0 = time()

    c_new = np.zeros((z_len, a_len))
    c_opt = np.zeros((z_len, a_len))

    a_new = np.zeros((z_len, a_len))
    a_opt = np.zeros((z_len, a_len))
    
    index = range(a_len)

    norm_pol = 1
    it = 0
    maxit = 5000

    # Initial guess for c_opt
    for j, z in enumerate(z_grid):
        c_opt[j, :] = z*w + r*a_grid[:]

    while norm_pol > 1e-6*(1-β) and it < maxit:
        
        ERHS = P @ (β*R * c_opt**(-σ))
        IERHS = ERHS**(-1/σ)
        
        for j, z in enumerate(z_grid):
            c_end = IERHS[j, :]
            a_end = (a_grid[:] + c_end - z*w) / R

            c_new[j, :] = InterpolatedUnivariateSpline(a_end, c_end, k=1)(a_grid)

            # Binding constraint
            bind = np.where(a_grid < a_end[0])
            c_new[j, bind] = R*a_grid[bind] + z*w - a_min

            a_new[j, :] = R*a_grid + z*w - c_new[j, :]

        norm_pol = np.max(np.abs(a_opt - a_new))
        c_opt[:, :] = c_new[:, :]
        a_opt[:, :] = a_new[:, :]

        it = it + 1

#     t1 = time()

#     print('Elapsed time is', t1-t0, 'seconds.')
#     print('Converged after', it, 'iterations.')
#     print('Policy function L_inf norm is', norm_pol)

#     t0 = time()
    
#     c_EGM = c_opt[:, :]
    
    a_grid_dens = np.linspace(a_min, a_max, dens_mult*a_len)
    a_len_dens = len(a_grid_dens)
    index_dens = range(a_len_dens)
    
    pos_l = np.zeros((z_len, a_len_dens), dtype=int)
    pos_h = np.zeros((z_len, a_len_dens), dtype=int)

    val_l = np.zeros((z_len, a_len_dens))
    val_h = np.zeros((z_len, a_len_dens))
    
    a_EGM = np.zeros((z_len, a_len_dens))

    for j, z in enumerate(z_grid):
        a_EGM[j, :] = InterpolatedUnivariateSpline(a_grid, a_opt[j, :], k=1)(a_grid_dens)

    for j, z in enumerate(z_grid):
        for i, a in enumerate(a_grid_dens):
            pos_h[j, i] = min(bisect(a_grid_dens, a_EGM[j, i]), a_len_dens-1)
            pos_l[j, i] = max(pos_h[j, i]-1, 0)

            val_h[j, i] = min(1, ((a_EGM[j, i] - a_grid_dens[pos_l[j, i]]) 
                                  / (a_grid_dens[pos_h[j, i]] - a_grid_dens[pos_l[j, i]])) )
            val_l[j, i] = 1 - val_h[j, i]

    G_0 = sparse.coo_matrix( ( np.concatenate((val_l[0, :], val_h[0, :])), 
                              ( np.concatenate((index_dens, index_dens)), 
                                np.concatenate((pos_l[0, :], pos_h[0, :])) ) ), 
                            shape=(a_len_dens, a_len_dens) ).tocsr()

    G_1 = sparse.coo_matrix( ( np.concatenate((val_l[1, :], val_h[1, :])), 
                              ( np.concatenate((index_dens, index_dens)), 
                                np.concatenate((pos_l[1, :], pos_h[1, :])) ) ), 
                            shape=(a_len_dens, a_len_dens) ).tocsr()

    G = sparse.vstack( (sparse.kron(P[0, :], G_0), 
                        sparse.kron(P[1, :], G_1)) )

    val, vec = eigs(G.transpose(), k=1, which='LR', return_eigenvectors=True)

#     print(val)

    stat_EGM = np.real(vec.flat)
    stat_EGM = stat_EGM / np.sum(stat_EGM)
    
    K_sup = a_grid_dens @ stat_EGM[a_len_dens:] + a_grid_dens @ stat_EGM[:a_len_dens]
    return K_sup

In [ ]:
rr = np.linspace(0, 0.04, 100) #1/β-1

KK = {}

for i, r in enumerate(rr):
    KK[i] = K_sup(r)

In [ ]:
# rr = np.linspace(0, 0.1, 100) #1/β-1

plt.plot(Capital_Demand(rr), rr*100, lw=2, label='Demand for capital')
plt.plot(KK.values(), rr*100, lw=2, label='Supply of capital')
plt.xlabel('Aggregate capital stock $K$')
plt.ylabel('Real interest rate $r$ (%)')
plt.legend(frameon=False, loc='upper center')
plt.xlim(None, 10.5)
plt.tight_layout()
# plt.savefig('Capital_eq.pdf')
plt.show()

In [ ]:
def Capital_Market(r, dens_mult=1, distribution=False, xlim=None):
    K_dem = (α*A/(r+δ))**(1/(1-α)) * N
    w = (1-α) * (K_dem/N)**α
    R = 1+r
    
    print('Interest rate pa =', r) #100*((1+r)**8-1)
    print('Capital demanded =', K_dem)
    
    # Endogenous Gridpoint Method

#     t0 = time()

    c_new = np.zeros((z_len, a_len))
    c_opt = np.zeros((z_len, a_len))

    a_new = np.zeros((z_len, a_len))
    a_opt = np.zeros((z_len, a_len))
    
    index = range(a_len)

    norm_pol = 1
    it = 0
    maxit = 5000

    # Initial guess for c_opt
    for j, z in enumerate(z_grid):
        c_opt[j, :] = z*w + r*a_grid[:]

    while norm_pol > 1e-6*(1-β) and it < maxit:
        
        ERHS = P @ (β*R * c_opt**(-σ))
        IERHS = ERHS**(-1/σ)
        
        for j, z in enumerate(z_grid):
            c_end = IERHS[j, :]
            a_end = (a_grid[:] + c_end - z*w) / R

            c_new[j, :] = InterpolatedUnivariateSpline(a_end, c_end, k=1)(a_grid)

            # Binding constraint
            bind = np.where(a_grid < a_end[0])
            c_new[j, bind] = R*a_grid[bind] + z*w - a_min

            a_new[j, :] = R*a_grid + z*w - c_new[j, :]

        norm_pol = np.max(np.abs(a_opt - a_new))
        c_opt[:, :] = c_new[:, :]
        a_opt[:, :] = a_new[:, :]

        it = it + 1

#     t1 = time()

#     print('Elapsed time is', t1-t0, 'seconds.')
#     print('Converged after', it, 'iterations.')
#     print('Policy function L_inf norm is', norm_pol)

#     t0 = time()
    
#     c_EGM = c_opt[:, :]
    
    a_grid_dens = np.linspace(a_min, a_max, dens_mult*a_len)
    a_len_dens = len(a_grid_dens)
    index_dens = range(a_len_dens)
    
    pos_l = np.zeros((z_len, a_len_dens), dtype=int)
    pos_h = np.zeros((z_len, a_len_dens), dtype=int)

    val_l = np.zeros((z_len, a_len_dens))
    val_h = np.zeros((z_len, a_len_dens))
    
    a_EGM = np.zeros((z_len, a_len_dens))

    for j, z in enumerate(z_grid):
        a_EGM[j, :] = InterpolatedUnivariateSpline(a_grid, a_opt[j, :], k=1)(a_grid_dens)

    for j, z in enumerate(z_grid):
        for i, a in enumerate(a_grid_dens):
            pos_h[j, i] = min(bisect(a_grid_dens, a_EGM[j, i]), a_len_dens-1)
            pos_l[j, i] = max(pos_h[j, i]-1, 0)

            val_h[j, i] = min(1, ((a_EGM[j, i] - a_grid_dens[pos_l[j, i]]) 
                                  / (a_grid_dens[pos_h[j, i]] - a_grid_dens[pos_l[j, i]])) )
            val_l[j, i] = 1 - val_h[j, i]

    G_0 = sparse.coo_matrix( ( np.concatenate((val_l[0, :], val_h[0, :])), 
                              ( np.concatenate((index_dens, index_dens)), 
                                np.concatenate((pos_l[0, :], pos_h[0, :])) ) ), 
                            shape=(a_len_dens, a_len_dens) ).tocsr()

    G_1 = sparse.coo_matrix( ( np.concatenate((val_l[1, :], val_h[1, :])), 
                              ( np.concatenate((index_dens, index_dens)), 
                                np.concatenate((pos_l[1, :], pos_h[1, :])) ) ), 
                            shape=(a_len_dens, a_len_dens) ).tocsr()

    G = sparse.vstack( (sparse.kron(P[0, :], G_0), 
                        sparse.kron(P[1, :], G_1)) )

    val, vec = eigs(G.transpose(), k=1, which='LR', return_eigenvectors=True)

#     print(val)

    stat_EGM = np.real(vec.flat)
    stat_EGM = stat_EGM / np.sum(stat_EGM)
    
    K_sup = a_grid_dens @ stat_EGM[a_len_dens:] + a_grid_dens @ stat_EGM[:a_len_dens]

#     t1 = time()

    print('Capital supplied =', K_sup)
    print('')
#     print('Elapsed time is', t1-t0, 'seconds.')

    if distribution == True:
        plt.plot(a_grid_dens, stat_EGM[a_len_dens:], lw=2, label='employed')
        plt.plot(a_grid_dens, stat_EGM[:a_len_dens], lw=2, label='unemployed')
        plt.title('Wealth distribution')
        plt.xlabel('Current period assets $a$')
        plt.xlim(xlim)
        plt.legend(frameon=False)
        plt.tight_layout()
#         plt.savefig('GE_dist.pdf')
        plt.show()
    
    return (K_sup - K_dem)

In [ ]:
Capital_Market(0.02, True)

In [ ]:
t0 = time()
result = optimize.brentq(Capital_Market, 0, 1/β-1, full_output=True)
t1 = time()
print('Elapsed time is', t1-t0, 'seconds.')

In [ ]:
r_star = result[0]

Capital_Market(result[0], 1, True)

In [ ]:
stat_EGM_both = stat_EGM[:a_len_dens] + stat_EGM[a_len_dens:]

stat_EGM_both[np.abs(stat_EGM_both) < 1e-15] = 0
stat_EGM_both = stat_EGM_both/sum(stat_EGM_both)

plt.plot(a_grid_dens, stat_EGM_both)
plt.show()

In [ ]:
a_dist_both = a_grid_dens * stat_EGM_both
a_dist_both = a_dist_both / sum(a_dist_both)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

plt.plot(np.linspace(0,1), np.linspace(0,1), lw=2, label='Perfect wealth equality')
plt.plot(np.insert(np.cumsum(stat_EGM_both), 0, 0), 
         np.insert(np.cumsum(a_dist_both), 0, 0), 
         lw=2, label='Lorenz curve')

plt.xlabel('Cumulative population')
plt.ylabel('Cumulative wealth')
plt.legend()

# plt.plot(P_bar[0], P_bar[0]*z_grid[0]/t_inc, 'o')
# plt.tight_layout()
# plt.savefig('GE_Lorenz.pdf')
plt.show()